In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
import pandas as pd
import seaborn as sns
import tensorflow as tf
#tfd = tf.contrib.distributions
import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors
layers = tf.contrib.layers
tf.set_random_seed(0)
from hepmc.core.densities.qcd import ee_qq_ng, export_hepmc, import_hepmc
from hepmc.core.densities.nice import Nice
from hepmc.core.phase_space.rambo import RamboOnDiet
from hepmc.core.phase_space.mapping import MappedDensity
from hepmc.core.sampling import Sample
from hepmc.core.integration.importance import ImportanceMC
from hepmc.core.sampling import Sample, AcceptRejectSampler
#from hepmc.core.markov.metropolis import DefaultMetropolis
#from hepmc.core.proposals import Gaussian

In [ ]:
eeqq2g = ee_qq_ng(2, 100., 5., .3)
rambo_mapping = RamboOnDiet(100., 4)
mapped = MappedDensity(eeqq2g, rambo_mapping)
training_sample = import_hepmc('../samples/qcd/2-4/training.hepmc')
training_sample = Sample(data=rambo_mapping.map_inverse(training_sample.data), weights=training_sample.weights)

In [ ]:
data = pd.DataFrame(training_sample.data)
f = sns.PairGrid(data)
f = f.map_diag(plt.hist, bins=15)
f = f.map_offdiag(plt.hist2d, bins=15)

In [ ]:
sherpa_weighted_sample = import_hepmc('../samples/qcd/2-4/sherpa_weighted.hepmc')
sherpa_weighted_sample = Sample(data=rambo_mapping.map_inverse(sherpa_weighted_sample.data), weights=sherpa_weighted_sample.weights)

In [ ]:
data = pd.DataFrame(sherpa_weighted_sample.data)
f = sns.PairGrid(data)
f = f.map_diag(plt.hist, bins=15)
f = f.map_offdiag(plt.hist2d, bins=15)

In [ ]:
%time nice = Nice(training_sample, train_iters=2000, num_bijectors=12, hidden_layers=[128, 128])

In [ ]:
sample = nice.rvs(100000)

In [ ]:
data = pd.DataFrame(sample)
f = sns.PairGrid(data)
f = f.map_diag(plt.hist, bins=15)
f = f.map_offdiag(plt.hist2d, bins=15, range=[[0, 1], [0, 1]])

In [ ]:
importance_sampler = ImportanceMC(mapped, nice)
%time nice_sample = importance_sampler.sample(100000)

In [ ]:
mapped_sample = Sample(data=rambo_mapping.map(nice_sample.data), weights=nice_sample.weights)
export_hepmc(100., mapped_sample, "../samples/qcd/2-4/realnvp.hepmc")

In [ ]:
bound = nice_sample.weights.max()
print(bound)

In [ ]:
nice_sample.weights.mean()/bound

In [ ]:
sampler = AcceptRejectSampler(target=mapped, bound=bound, sampler=importance_sampler, sampling_pdf=nice.pdf)

In [ ]:
%time sample = sampler.sample(10000, expected_efficiency=.07)

In [ ]:
class AcceptRejectSampler(object):
    """ Acceptance Rejection method for sampling a given pdf.
    
    The method uses a known distribution and sampling method to propose
    samples which are then accepted with the probability
    pdf(x)/(c * sampling_pdf(x)), thus producing the desired distribution. 
    The resulting sample is unweighted.
    
    .. todo::
        Handle points that lie above the bound.
    """

    def __init__(self, target, bound: float, 
            sampler = None, sampling_pdf = None) -> None:
        """
        Parameters
        ----------
        target
            Unnormalized desired probability distribution of the sample.
        bound
            Constant such that pdf(x) <= bound * sampling_pdf(x)
            for all x in the range of sampling.
        sampler
            The sampler which generates the sample. The default is a uniform sampler.
        """
        self.target = target
        self.bound = bound
        self.ndim = target.ndim

        if sampler is None:
            sampler = UniformSampler(target)
            def sampling_pdf(x):
                return np.ones(x.size)

        self.sampler = sampler
        self.sampling_pdf = sampling_pdf

    def sample(self, sample_size: int, expected_efficiency: float = 1.) -> None:
        x = np.empty((sample_size, self.ndim))

        #indices = np.arange(sample_size)
        n_todo = sample_size
        trials = 0
        while n_todo > 0:
            print('n_todo:', n_todo)
            trials += int(n_todo/expected_efficiency)
            sample = self.sampler.sample(int(n_todo/expected_efficiency))
            proposal = sample.data
            #accept = np.random.rand(indices.size) * self.bound * self.sampling_pdf(proposal) <= self.target.pdf(proposal)
            #accept = np.random.rand(indices.size) * self.bound <= sample.weights
            #accept = np.random.rand(indices.size) < sample.weights / self.bound
            u = np.random.rand(int(n_todo/expected_efficiency))
            accept = u < sample.weights / self.bound
            n_accept = accept.sum()
            if n_accept <= n_todo:
                x[sample_size-n_todo:sample_size-n_todo+n_accept] = proposal[accept]
            else:
                accepted = proposal[accept]
                x[sample_size-n_todo:] = accepted[:n_todo]
            n_todo -= n_accept
            #x[indices[accept]] = proposal[accept]
            #indices = indices[np.logical_not(accept)]
        print('Unweighting eff.:', sample_size/trials)
        return Sample(data=x, target=self.target)

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
DTYPE=tf.float32
NP_DTYPE=np.float32
USE_BATCHNORM = False

In [ ]:
batch_size = 100
num_bijectors = 5
train_iters = 1e4

In [ ]:
mapped_sample = Sample(data=rambo_mapping.map(sample.data), weights=sample.weights)
export_hepmc(100., mapped_sample, "../samples/qcd/2-4/realnvp.hepmc")